### Circle Center Detection Function

The circle center detection is based on a **normal intersection voting method**:

1. **Edge Extraction**: First, we extract the edges of the cleaned binary image using Canny edge detection to identify potential boundary points of the circular pattern.

2. **Gradient Calculation**: For each edge pixel, we compute the gradient vector (using Sobel operators) which represents the direction of intensity change and helps determine the edge normal direction.

3. **Voting along Normals**: Each edge point votes for possible circle centers by projecting votes along the normal directions (both inward and outward) within a defined radius range. This accumulates votes in an accumulator space.

4. **Finding the Peak Vote**: The pixel with the highest vote count in the accumulator is selected as the estimated circle center, since normals from many edge points intersect there.

This method is robust to noise and slight shape distortions, making it suitable for detecting approximate geometric centers in interference fringe patterns or similar circular features.


After multiple rounds of testing and parameter tuning, this voting-based circle center detection method consistently delivers reliable and accurate results. The accumulator heatmap clearly highlights the peak vote region, and the detected center aligns well with the visual center of the circular patterns.

The following figures illustrate typical detection outcomes, demonstrating the method’s robustness and effectiveness across varied samples and noise conditions.

![result](./image/finder.png)

### Edge Detection (Canny)

- Use the Canny edge detector to extract edge points from the cleaned binary image.
- These edge points are candidates for the circle boundary.
- Parameters 50 and 150 are thresholds for edge linking and hysteresis.

### Gradient Calculation (Sobel)

- Calculate the gradient in X and Y directions to estimate the edge orientation.
- These gradients are essential for computing the **normal vectors** at edge points.
- Kernel size 5 smooths gradients to reduce noise influence.


### Accumulator Initialization

- Create a 2D accumulator array matching the image size to store votes.
- Each pixel in this array will accumulate votes from edge points pointing towards possible centers.


### Edge Points Extraction

- Extract coordinates `(y, x)` of all edge pixels detected by Canny.
- These points will be used to cast votes in the accumulator.


### Voting Along Normals

- For each edge point, compute the normalized gradient vector (normal to the edge).
- Cast votes along the positive and negative normal directions for possible centers.
- Voting range is from `min_radius` to `max_radius`, stepping pixel by pixel.
- Each valid accumulator pixel gets one vote.
- This accumulates evidence of circle centers where normals from multiple edge points intersect.


### Detecting the Peak in Accumulator

- Find the location with the highest vote count in the accumulator.
- This location corresponds to the most likely circle center.


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def find_circle_center_voting(clean_img, min_radius=10, max_radius=None):
    if max_radius is None:
        max_radius = int(min(clean_img.shape) * 0.5)  #70%

    edges = cv2.Canny(clean_img, 50, 150)

    sobel_x = cv2.Sobel(clean_img, cv2.CV_64F, 1, 0, ksize=5)
    sobel_y = cv2.Sobel(clean_img, cv2.CV_64F, 0, 1, ksize=5)

    accumulator = np.zeros_like(clean_img, dtype=np.float32)

    edge_points = np.column_stack(np.where(edges > 0))

    #vote
    for y, x in edge_points:
        # Gradient Vector
        gx = sobel_x[y, x]
        gy = sobel_y[y, x]

        mag = np.sqrt(gx ** 2 + gy ** 2)

        if mag < 1e-5:
            continue
        
        # Normalization
        nx = gx / mag
        ny = gy / mag

        for direction in [-1, 1]:

            for r in range(min_radius, max_radius + 1): 
                cx = int(x + direction * r * nx)
                cy = int(y + direction * r * ny)

                if 0 <= cy < accumulator.shape[0] and 0 <= cx < accumulator.shape[1]:
                    accumulator[cy, cx] += 1

    # find center(max of voting result)
    _, _, _, max_loc = cv2.minMaxLoc(accumulator)
    center_x, center_y = max_loc

    plt.figure(figsize=(10, 5))
    plt.subplot(121), plt.imshow(accumulator, cmap='hot'), plt.title('Accumulator Space')
    plt.subplot(122), plt.imshow(cv2.cvtColor(clean_img, cv2.COLOR_GRAY2RGB))
    plt.scatter(center_x, center_y, s=100, c='red', marker='+')
    plt.title('Detected Center')
    plt.show()

    return center_x, center_y